# Qiskit Comparison
This notebook highlights how my simulator is a nice little subset of qiskit's simulator.

## Qiskit Circuit
To validate my implementation, first we create a simple circuit in qiskit and run simluation on it.

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(5, 5)
qc.h(0)
qc.h(1)
qc.cz(0, 4)
qc.unitary([[0.5 + 0.5j, 0.5 - 0.5j], [0.5 - 0.5j, 0.5 + 0.5j]], 1)
out = qc.measure([0, 1, 2, 3, 4], [0, 1, 2, 3, 4])
qc.draw()


┌───┐              ┌─┐      
q_0: ┤ H ├────────────■─┤M├──────
     ├───┤┌─────────┐ │ └╥┘┌─┐   
q_1: ┤ H ├┤ unitary ├─┼──╫─┤M├───
     └┬─┬┘└─────────┘ │  ║ └╥┘   
q_2: ─┤M├─────────────┼──╫──╫────
      └╥┘     ┌─┐     │  ║  ║    
q_3: ──╫──────┤M├─────┼──╫──╫────
       ║      └╥┘     │  ║  ║ ┌─┐
q_4: ──╫───────╫──────■──╫──╫─┤M├
       ║       ║         ║  ║ └╥┘
c: 5/══╩═══════╩═════════╩══╩══╩═
       2       3         0  1  4

In [2]:
from qiskit import Aer, execute
import time

def simulate(circuit, num_shots):
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(circuit, backend_sim, shots=num_shots)
    result_sim = job_sim.result()
    return result_sim.get_counts(circuit)

start = time.time()
qiskit_counts = simulate(qc, num_shots=10000)
qiskit_diff = time.time() - start

## JQiskit Circuit
Then I built the same circuit in my simulator and ran it.

In [3]:
from jqiskit.api import QuantumCircuit as JQuantumCircuit

qc = JQuantumCircuit(5)
qc.h(0)
qc.h(1)
qc.cz(0, 4)
qc.sqrtnot(1)

start = time.time()
jqiskit_counts, state = qc.measure(num_shots=10000)
jqiskit_diff = time.time() - start

## Timing Comparison

This clearly isn't a fair fight, as qiskit is probably doing a bunch of stuff under the hood for more complex features that I'm not doing for this simple simulator. But for what it's worth the simple version is significantly faster for the circuit described above. This may indicate a need for a more light-weight framework in the future for faster hyper parameter searches and stuff like that.

In [4]:
print(f'jqiskit_speed: {jqiskit_diff * 1000:.2f} ms')
print(f' qiskit_speed: {qiskit_diff * 1000:.2f} ms')

jqiskit_speed: 6.23 ms
 qiskit_speed: 127.32 ms


## Validation

I then compared the two by looking at the simulation output states. Note how qiskit is the opposite endian-ness as my implementation, so I had to flip the state to do a proper comparison.

In [5]:
assert len(jqiskit_counts) == len(qiskit_counts), "Number of states don't match!"

for state_str in jqiskit_counts:
    print(f'state: {state_str}, qiskit: {qiskit_counts[state_str[::-1]]}; jqiskit: {jqiskit_counts[state_str]}')

state: 11000, qiskit: 2515; jqiskit: 2449
state: 01000, qiskit: 2478; jqiskit: 2499
state: 10000, qiskit: 2467; jqiskit: 2507
state: 00000, qiskit: 2540; jqiskit: 2545


In [6]:
import numpy as np

qc = JQuantumCircuit(2)
qc.parametric('[[1, 0.], [0.0 , exp(1.0j*theta)]]', 0)
theta = np.pi / 7
qc.measure(feed_dict={'theta': theta})

1


({'00': 1000}, array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]))

In [7]:
qc = QuantumCircuit(2, 2)
qc.unitary([[1, 0.], [0.0, np.exp(1.0j*theta)]], 0)
qc.measure([0, 1], [0, 1])
simulate(qc, 10000)

{'00': 10000}